In [1]:
import csv
import random
import pandas as pd
import numpy as np

In [2]:
"""Bootstrapping Data"""
def difftables(dataset,rows,coloums,target):  #bootstrap and subspacing
    td = pd.read_csv(dataset)
    tot_no_rows = len(td.index)
    tot_no_col = len(td.columns)
    rows = np.random.randint(tot_no_rows, size=(rows))  #bootstraping
    index = np.random.randint(tot_no_col-1, size=(coloums))
    return td.iloc[rows,list(index)].join(td.iloc[rows,[target]])

In [3]:
class HelperFunctions:
    def __init__(self):
        pass
    
    """Find the unique values for a column in a dataset."""
    def unique_vals(self, rows, col):
        return set([row[col] for row in rows])
    
    """Count No of each Unique Val for the prediction."""
    def class_counts(self, rows):
        counts = {}
        for row in rows:
            label = row[-1]
            if label not in counts:
                counts[label] = 0
            counts[label] += 1
        return counts
    
    """Check whether a value is numeric or not"""
    def is_numeric(self, value):
        return isinstance(value, int) or isinstance(value, float)
    
    def partition(self, rows, question):
        """
        Partitions a dataset. For each row in the dataset, 
        check if it matches the question. If so, add it to 
        'true rows', otherwise, add it to 'false rows'.
        """
        true_rows, false_rows = [], []
        for row in rows:
            if question.match(row):
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows

In [4]:
class Question:
    """A Question is used to partition a dataset."""

    def __init__(self, column, value, unique_count = None):
        self.column = column
        self.value = value
        self.unique_count = helper.unique_vals(training_data, column)

    def match(self, example):
        val = example[self.column]
        if len(self.unique_count)<=10:
            return val == self.value
        else:
            return val >= self.value

    def __repr__(self):
        # Printing the question in a readable format.
        condition = "=="
        if len(self.unique_count)>10:
            condition = ">="
        return "if %s %s %s:" % (
            header[self.column], condition, str(self.value))

In [5]:
def gini(rows):
    """Finding the impurities remaining in a node"""
    counts = helper.class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainty):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [6]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = helper.partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [7]:
class Leaf:
    def __init__(self, rows):
        self.predictions = helper.class_counts(rows)

In [8]:
class Decision_Node:
    """
    A Decision Node asks a question.
    This holds a reference to the question, 
    and to the two child nodes.
    """

    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [9]:
def build_tree(rows):
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain, we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = helper.partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [10]:
# def print_tree(node, spacing=""):
#     # Base case: we've reached a leaf
#     if isinstance(node, Leaf):
#         print (spacing + "Predict", node.predictions)
#         return

#     # Print the question at this node
#     print (spacing + str(node.question))

#     # Call this function recursively on the true branch
#     print (spacing + '--> True:')
#     print_tree(node.true_branch, spacing + "    ")

#     # Call this function recursively on the false branch
#     print (spacing + '--> False:')
#     print_tree(node.false_branch, spacing + "    ")

In [11]:
def classify(row, node):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [12]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100))
    return probs

In [13]:
"""Accuracy of the Model on scale 0 to 1"""
def accuracy(y_true,y_pred,normalize=True):
    accuracy=[]
    for i in range(len(y_pred)):
        if y_pred[i]==y_true[i]:
            accuracy.append(1)
        else:
            accuracy.append(0)
    if normalize==True:
        return np.mean(accuracy)
    if normalize==False:
        return sum(accuracy)

In [14]:
def save_tree(node, spacing="    "):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        key = int(list(node.predictions.keys())[0])
        if key==1:
            f.write(f"{spacing}count_yes+=1\n")
        else:
            f.write(f"{spacing}count_no+=1\n")
        return

    # Print the question at this node in the file
    f.write(spacing + str(node.question) + '\n')

    # Call this function recursively on the true branch
    save_tree(node.true_branch, spacing + "    ")

    # Call this function recursively on the false branch
    f.write(spacing + 'else:\n')
    save_tree(node.false_branch, spacing + "    ")

In [15]:
"""Building 100 trees and saving it locally in a .py file"""

# path = "E:/AIml proj/tree.py"
with open('tree.py','w') as f:
    f.write(str(f'def tree(data,count_yes,count_no):\n'))  # First Line
    for loob in range(100):
        # Importing Data from heart.csv file
        df = difftables('heart.csv',200,9,13)
        df = df.T.drop_duplicates().T
        
        # Storing all Headers / Column Labels
        header = []
        for col in df.columns:
            header.append(col)
        new_header = []
        for item in header:
            new_header.append(f"data['{item}']")
        header = new_header[:]
        
        # Getting Training Dataset Ready
        data = df.to_numpy().tolist()
        size = len(data)
        training_data = data 
        
        # Building a Tree from the Randomly Selected Samples
        helper = HelperFunctions()
        best_gain, best_question = find_best_split(training_data)
        my_tree = build_tree(training_data)

        # Storing the tree locally into tree.py file
        save_tree(my_tree)  
        
    # Returning the prediction
    f.write('\n    if count_yes >= count_no: return 1\n    else: return 0\n\n')

In [16]:
"""Getting the Testing Dataset"""

with open('Heart.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
# Randomize
random.shuffle(data)

testing_data = []
# Storing Correct Target Values
actual = []
for i in range(0, 40):
    actual.append(int(data[i][-1]))
    testing_data.append(data[i][:-1])

In [17]:
"""Testing the model and storing predictions into pred list"""

# import sys
# sys.path.append('E:/AIml proj')
import tree as tree

pred = []
for g in testing_data:
    data_patient = {'age':0,'sex':0,'cp':0,'trestbps':0,'chol':0,'fbs':0,'restecg':0,
                    'thalach':0,'exang':0,'oldpeak':0,'slope':0,'ca':0,'thal':0}
    data_patient['age'] = float(g[0])
    data_patient['sex'] = float(g[1])
    data_patient['cp'] = float(g[2])
    data_patient['trestbps'] = float(g[3])
    data_patient['chol'] = float(g[4])    
    data_patient['fbs'] = float(g[5])
    data_patient['restecg'] = float(g[6])
    data_patient['thalach'] = float(g[7])
    data_patient['exang'] = float(g[8])    
    data_patient['oldpeak'] = float(g[9])
    data_patient['slope'] = float(g[10])
    data_patient['ca'] = float(g[11])
    data_patient['thal'] = float(g[12])
    pred.append(tree.tree(data_patient, 0, 0))

In [18]:
# Checking Accuracy
print(accuracy(actual,pred))

0.975
